first we import all the neccessary packages for working with the data

In [11]:
import os
from IPython.display import display
#import hoboreader
import pandas as pd
import shutil
import matplotlib as mp
import re

next we ensure that our working directory is set correctly so that when we're working with file paths they always work relative to our project

In [2]:
#os.chdir("..")
os.chdir("C:\\Users\\samer\\OneDrive - Cal Poly\\Classes\\SPR_Fellowship\\SPR_Fellowship")
os.getcwd()

'c:\\Users\\samer\\OneDrive - Cal Poly\\Classes\\SPR_Fellowship\\SPR_Fellowship'

First well create a function for searching through complex fill structures and returning the paths to files of only a certain file type

In [3]:
def search_filetype(path = ".", extentension = ""):
    """
    :param path: string, where to search
    :param extentension: string, what files to look for
    :return: list, paths to all files with a matching extension
    """
    paths = []
    for dirpath, dirnames, filenames in os.walk(path):
        for filename in filenames:
            if filename[-len(extentension):] == extentension:
                paths.append(dirpath + "/" + filename)
    return paths

Well also want some test data to work with along the way before we start working with thousands of files

In [4]:
list_ex_headers = search_filetype(path="LegacyFiles/All_txt/txt_header_examples/", extentension=".txt")

print(list_ex_headers)

['LegacyFiles/All_txt/txt_header_examples//Al RRG_030916.txt', 'LegacyFiles/All_txt/txt_header_examples//AL RRG_060403a.txt', 'LegacyFiles/All_txt/txt_header_examples//AL_RRG_091222 remove first tip.txt']


# Dealing with Headers
Next we create a function for working with the messsy headers created by the text files.  There are a few different ones to worry about, we have to manage for each one, while preserving inprotant information, like file name and serial numbers.

In [8]:
def header_fixer(path = "."):
    """
    :param path:
    :param col_names:
    :return:
    """
    csv = pd.read_csv(path, sep="\t") #read in the csv just to check its structure
    
    if csv.shape[1] < 2:  #because files with SN have a multiline header this will filter those files out
        serial_Num = re.findall("[0-9]{5,6}", csv.columns[0])
        csv = pd.read_csv(path, header=1, sep="\t")
        csv["Serial Number"] = serial_Num[0]
    else:
        csv = pd.read_csv(path, sep="\t")
        csv["Serial Number"] =  None

    return csv


great, now lets test it out on our small sample of txt files we created before

In [18]:

ex_headers_csv_list = []

for path in list_ex_headers:
    csv = header_fixer(path = path)
    csv["FileName"] = path.split(sep="/")[-1]
    ex_headers_csv_list.append(csv) #create a list of test dataframes
    display(csv.head())

,Date Time,Event (Rain Bucket Tip),Serial Number,FileName
0,06/18/03 08:40:17.0,0.00,None,Al RRG_030916.txt
1,09/09/03 14:19:42.0,0.01,None,Al RRG_030916.txt
2,09/09/03 15:43:36.5,0.02,None,Al RRG_030916.txt
3,09/09/03 16:33:22.0,0.03,None,Al RRG_030916.txt
4,09/09/03 17:20:45.5,0.04,None,Al RRG_030916.txt


,Date Time,Event (Rain bucket tip),Serial Number,FileName
0,03/26/06 17:58:38.0,0.00,531980,AL RRG_060403a.txt
1,03/27/06 13:23:29.5,0.01,531980,AL RRG_060403a.txt
2,03/27/06 13:37:13.5,0.02,531980,AL RRG_060403a.txt
3,03/27/06 13:47:53.5,0.03,531980,AL RRG_060403a.txt
4,03/27/06 13:53:57.5,0.04,531980,AL RRG_060403a.txt


,Date Time,Event (Events),Serial Number,FileName
0,12/15/09 16:31:03.0,0.000000,None,AL_RRG_091222 remove first tip.txt
1,12/15/09 16:31:24.0,0.010870,None,AL_RRG_091222 remove first tip.txt
2,12/16/09 09:04:57.0,0.021739,None,AL_RRG_091222 remove first tip.txt
3,12/16/09 09:20:12.5,0.032609,None,AL_RRG_091222 remove first tip.txt
4,12/16/09 09:40:41.0,0.043478,None,AL_RRG_091222 remove first tip.txt


,Date Time,Event (Rain bucket tip),Serial Number,FileName
0,03/26/06 17:58:38.0,0.00,531980,AL RRG_060403a.txt
1,03/27/06 13:23:29.5,0.01,531980,AL RRG_060403a.txt
2,03/27/06 13:37:13.5,0.02,531980,AL RRG_060403a.txt
3,03/27/06 13:47:53.5,0.03,531980,AL RRG_060403a.txt
4,03/27/06 13:53:57.5,0.04,531980,AL RRG_060403a.txt
...,...,...,...,...
333,04/03/06 11:34:18.5,3.33,531980,AL RRG_060403a.txt
334,04/03/06 11:38:36.5,3.34,531980,AL RRG_060403a.txt
335,04/03/06 11:43:23.5,3.35,531980,AL RRG_060403a.txt
336,04/03/06 11:49:16.5,3.36,531980,AL RRG_060403a.txt


# deal with column names
That seems to be working okay, but there are definitely some differences in column names that are going to give us issues later, so lets create a function to deal with those. 

# deal with cummulative values
https://stackoverflow.com/questions/27581942/how-can-i-get-back-real-value-from-cumulative-sum-in-pandas

# combine all dataframes